In [496]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy.dialects.mssql.information_schema import columns
from sklearn.preprocessing import OneHotEncoder

# Experiment 8
In this experiment, apart from previous attempts, samples are to be split into train-test sets by the samples (Wood, Coal, Digestate, etc.), not random 80-20 split.

## Aim of the action
- As a result of regular 80-20 train-test-split approach, part of the data points from the same sample measurement at the same parameters fall into the test split (statistically, around one-third of the same sample at same conditions). This leads data leakage, and unreliable accuracy performance of model on test dataset.

## Details & Key Changes
### Outlier Removal
Biomass Data Before: 740 	After: 733	 Removed: 7
Mix Data Before:	 563 	After: 555	 Removed: 8
Coal Data Before:	 195 	After: 191	 Removed: 4
Plastic Data Before: 492 	After: 482	 Removed: 10

- test data contains **biomass, mix, plastic** fuels 'HTC, digestate_pe, ebs2)
- max r2: 0.7387 with LightGBM (Baselines w/o hyperparameter tuning)


## Comments | To be improved
- Other fuel combinations are to be selected as test set.

In [497]:
features = pd.read_csv(r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\preprocess\features.csv", delimiter=';')
labels = pd.read_csv(r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\preprocess\labels.csv", delimiter=';')

In [498]:
features.columns

Index(['sample', 'wc_1', 'vm_1', 'fc_1', 'ac_1', 'c_1', 'h_1', 'o_1', 'n_1',
       's_1', 'cl_1', 'hc_1', 'oc_1', 'lhv_1', 'wc_2', 'vm_2', 'fc_2', 'ac_2',
       'c_2', 'h_2', 'o_2', 'n_2', 's_2', 'cl_2', 'hc_2', 'oc_2', 'lhv_2',
       'atmosphere', 'temperature', 'residence_time', 'pressure', 'heat_rate',
       'x_fuel1', 'x_fuel2'],
      dtype='object')

In [499]:
wc = features['wc_1']*features['x_fuel1'] + features['wc_2']*features['x_fuel2']
vm = features['vm_1']*features['x_fuel1'] + features['vm_2']*features['x_fuel2']
fc = features['fc_1']*features['x_fuel1'] + features['fc_2']*features['x_fuel2']
ac = features['ac_1']*features['x_fuel1'] + features['ac_2']*features['x_fuel2']
c = features['c_1']*features['x_fuel1'] + features['c_2']*features['x_fuel2']
h = features['h_1']*features['x_fuel1'] + features['h_2']*features['x_fuel2']
o = features['o_1']*features['x_fuel1'] + features['o_2']*features['x_fuel2']
n = features['n_1']*features['x_fuel1'] + features['n_2']*features['x_fuel2']
s = features['s_1']*features['x_fuel1'] + features['s_2']*features['x_fuel2']
cl = features['cl_1']*features['x_fuel1'] + features['cl_2']*features['x_fuel2']
hc = features['hc_1']*features['x_fuel1'] + features['hc_2']*features['x_fuel2']
oc = features['oc_1']*features['x_fuel1'] + features['oc_2']*features['x_fuel2']
lhv = features['lhv_1']*features['x_fuel1'] + features['lhv_2']*features['x_fuel2']

In [500]:
features.drop(['wc_1', 'vm_1', 'fc_1', 'ac_1', 'c_1', 'h_1', 'o_1', 'n_1',
       's_1', 'cl_1', 'hc_1', 'oc_1', 'lhv_1', 'wc_2', 'vm_2', 'fc_2', 'ac_2',
       'c_2', 'h_2', 'o_2', 'n_2', 's_2', 'cl_2', 'hc_2', 'oc_2', 'lhv_2',
       'atmosphere','x_fuel1', 'x_fuel2'], axis=1, inplace=True)

In [501]:
features = pd.concat([features, wc, vm, fc, ac, c,h,o,n,s,cl,hc,oc,lhv ],  axis=1)


In [502]:
features.columns = ['sample', 'temperature', 'residence_time', 'pressure',
            'heat_rate', 'wc', 'vm', 'fc', 'ac', 'c', 'h', 'o', 'n', 's', 'cl', 'hc', 'oc', 'lhv']

In [503]:
features.head()

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,o,n,s,cl,hc,oc,lhv
0,brown_coal_1,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
1,brown_coal_2,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
2,brown_coal_3,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
3,brown_coal_4,600,0.5,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13
4,brown_coal_5,600,1.0,1.0,1000,10.8,44.2,40.9,4.2,63.4,3.9,16.2,0.9,0.6,0.0,0.7,0.2,20.13


In [504]:
def classify_fuel(sample):
    sample = str(sample).lower()

    if "gumm" in sample or "gm" in sample:
        return "gummipulver"
    elif "wod" in sample or 'wood' in sample or 'wd' in sample:
        return "wood"
    elif "coal" in sample or 'bks' in sample:
        return "coal"
    elif "swage" in sample or 'swg' in sample or 'sewage' in sample:
        return "sewage"
    elif "refuse" in sample or 'ref' in sample:
        return "ebs2"
    elif "digestate" in sample or 'dgst' in sample or "dgt" in sample:
        if 'pe' not in sample and 'pp' not in sample and 'car' not in sample:
            return "digestate"
        if 'pe' in sample:
            return "digestate_pe"
        if 'car' in sample:
            return "digestate_car"
        if 'pp' in sample:
            return "digestate_pp"
    elif "htc" in sample:
        return "HTC"
    elif "cel" in sample:
        return "cellulose"
    elif "hc" in sample:
        return "Hemicellulose"
    elif "lig" in sample:
        return "Lignin"
    elif "ebs2" in sample:
        return "ebs2"
    elif "ebs1" in sample:
        return "ebs1"
    else:
        return "unknown"  # Default category


features['fuel_type'] = features['sample'].apply(classify_fuel)

print(features.head())

         sample  temperature  residence_time  pressure  heat_rate    wc    vm  \
0  brown_coal_1          600             0.5       1.0       1000  10.8  44.2   
1  brown_coal_2          600             0.5       1.0       1000  10.8  44.2   
2  brown_coal_3          600             0.5       1.0       1000  10.8  44.2   
3  brown_coal_4          600             0.5       1.0       1000  10.8  44.2   
4  brown_coal_5          600             1.0       1.0       1000  10.8  44.2   

     fc   ac     c    h     o    n    s   cl   hc   oc    lhv fuel_type  
0  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
1  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
2  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
3  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  
4  40.9  4.2  63.4  3.9  16.2  0.9  0.6  0.0  0.7  0.2  20.13      coal  


In [505]:
def fuel_category(sample):
    sample = str(sample).lower()

    if "gumm" in sample or "gm" in sample:
        return "plastic"
    elif "wod" in sample or 'wood' in sample or 'wd' in sample:
        return "biomass"
    elif "coal" in sample or 'bks' in sample:
        return "coal"
    elif "swage" in sample or 'swg' in sample or 'sewage' in sample:
        return "biomass"
    elif "refuse" in sample or 'ref' in sample:
        return "plastic"
    elif "digestate" in sample or 'dgst' in sample or "dgt" in sample:
        if 'pe' not in sample and 'pp' not in sample and 'car' not in sample:
            return "plastic"
        if 'pe' in sample:
            return "mix"
        if 'car' in sample:
            return "mix"
        if 'pp' in sample:
            return "mix"
    elif "htc" in sample:
        return "biomass"
    elif "cel" in sample:
        return "biomass"
    elif "hc" in sample:
        return "biomass"
    elif "lig" in sample:
        return "biomass"
    elif "ebs2" in sample:
        return "plastic"
    elif "ebs1" in sample:
        return "plastic"
    else:
        return "unknown"  # Default category

features['fuel_category'] = features['sample'].apply(fuel_category)

In [506]:
set(features['fuel_category'])

{'biomass', 'coal', 'mix', 'plastic'}

In [507]:
features.head

<bound method NDFrame.head of             sample  temperature  residence_time  pressure  heat_rate    wc  \
0     brown_coal_1          600             0.5       1.0       1000  10.8   
1     brown_coal_2          600             0.5       1.0       1000  10.8   
2     brown_coal_3          600             0.5       1.0       1000  10.8   
3     brown_coal_4          600             0.5       1.0       1000  10.8   
4     brown_coal_5          600             1.0       1.0       1000  10.8   
...            ...          ...             ...       ...        ...   ...   
1985         gm_39         1000             1.0      10.0         10   2.6   
1986         gm_40         1000             1.0      10.0         50   2.6   
1987         gm_41         1000             1.0      10.0        100   2.6   
1988         gm_42         1000             1.0      10.0        500   2.6   
1989         gm_43         1000             1.0      10.0       1000   2.6   

        vm    fc    ac     c    h

In [508]:
features.isnull().sum()

sample              0
temperature         0
residence_time      0
pressure            0
heat_rate           0
wc                  0
vm                  0
fc                  0
ac                  0
c                   0
h                   0
o                   0
n                   0
s                   0
cl                  0
hc                  0
oc                  0
lhv               206
fuel_type           0
fuel_category       0
dtype: int64

In [509]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(features[['fuel_category']])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out())
features_encoded = pd.concat([features, one_hot_df], axis=1)
features_encoded.drop(columns=['fuel_category'], inplace=True)
print(features_encoded.head())


         sample  temperature  residence_time  pressure  heat_rate    wc    vm  \
0  brown_coal_1          600             0.5       1.0       1000  10.8  44.2   
1  brown_coal_2          600             0.5       1.0       1000  10.8  44.2   
2  brown_coal_3          600             0.5       1.0       1000  10.8  44.2   
3  brown_coal_4          600             0.5       1.0       1000  10.8  44.2   
4  brown_coal_5          600             1.0       1.0       1000  10.8  44.2   

     fc   ac     c  ...    s   cl   hc   oc    lhv  fuel_type  \
0  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
1  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
2  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
3  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   
4  40.9  4.2  63.4  ...  0.6  0.0  0.7  0.2  20.13       coal   

   fuel_category_biomass  fuel_category_coal fuel_category_mix  \
0                    0.0                 1.0            

In [510]:
features_encoded = pd.read_csv(r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\Data\!Experiment_1\features_encoded.csv")

In [511]:
fuel_type_counts = features_encoded['fuel_type'].value_counts()
print(fuel_type_counts)

fuel_type
sewage           241
digestate        236
wood             236
digestate_car    206
coal             195
digestate_pp     183
digestate_pe     174
ebs2             138
HTC              134
ebs1              62
gummipulver       56
Hemicellulose     44
Lignin            44
cellulose         41
Name: count, dtype: int64


In [512]:
data = pd.concat([features_encoded, labels['devol_yield']], axis=1)

In [513]:
data.columns

Index(['sample', 'temperature', 'residence_time', 'pressure', 'heat_rate',
       'wc', 'vm', 'fc', 'ac', 'c', 'h', 'o', 'n', 's', 'cl', 'hc', 'oc',
       'lhv', 'fuel_type', 'fuel_category_biomass', 'fuel_category_coal',
       'fuel_category_mix', 'fuel_category_plastic', 'devol_yield'],
      dtype='object')

In [514]:
def remove_outliers_iqr(group, cols, multiplier=1.5):
    # If cols is a string, convert it to a list
    if isinstance(cols, str):
        cols = [cols]
    Q1 = group[cols].quantile(0.25)
    Q3 = group[cols].quantile(0.75)
    IQR = Q3 - Q1
    # Keep rows where all specified columns are within the acceptable range
    mask = ~((group[cols] < (Q1 - multiplier * IQR)) | (group[cols] > (Q3 + multiplier * IQR))).any(axis=1)
    return group[mask]

# Assuming `data` is your DataFrame:
biomass_data = data[data['fuel_category_biomass'] == 1]
mix_data = data[data['fuel_category_mix'] == 1]
coal_data = data[data['fuel_category_coal'] == 1]
plastic_data = data[data['fuel_category_plastic'] == 1]

# Group each category by 'temperature'
biomass = biomass_data.groupby('temperature')
mix = mix_data.groupby('temperature')
coal = coal_data.groupby('temperature')
plastic = plastic_data.groupby('temperature')

biomass_clean = biomass.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
mix_clean = mix.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
coal_clean = coal.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
plastic_clean = plastic.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)

data_clean = data.groupby('temperature').apply(lambda g: remove_outliers_iqr(g, 'devol_yield'))

C:\Users\demir\AppData\Local\Temp\ipykernel_8792\3437265364.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  biomass_clean = biomass.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index(drop=True)
C:\Users\demir\AppData\Local\Temp\ipykernel_8792\3437265364.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mix_clean = mix.apply(lambda g: remove_outliers_iqr(g, 'devol_yield')).reset_index

In [515]:
print(f'Biomass Data Before: {biomass_data.shape[0]}', f'\tAfter: {biomass_clean.shape[0]}\t Removed: {biomass_data.shape[0] - biomass_clean.shape[0]}')
print(f'Mix Data Before:\t {mix_data.shape[0]}', f'\tAfter: {mix_clean.shape[0]}\t Removed: {mix_data.shape[0] - mix_clean.shape[0]}')
print(f'Coal Data Before:\t {coal_data.shape[0]}', f'\tAfter: {coal_clean.shape[0]}\t Removed: {coal_data.shape[0] - coal_clean.shape[0]}')
print(f'Plastic Data Before: {plastic_data.shape[0]}', f'\tAfter: {plastic_clean.shape[0]}\t Removed: {plastic_data.shape[0] - plastic_clean.shape[0]}')

Biomass Data Before: 740 	After: 733	 Removed: 7
Mix Data Before:	 563 	After: 555	 Removed: 8
Coal Data Before:	 195 	After: 191	 Removed: 4
Plastic Data Before: 492 	After: 482	 Removed: 10


In [516]:
clean_data = pd.concat([biomass_clean, mix_clean, coal_clean, plastic_clean])
clean_data.head()

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,...,cl,hc,oc,lhv,fuel_type,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
0,htc_96,200,0.5,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.000000,1.200000,0.40000,13.528000,HTC,1.0,0.0,0.0,0.0,6.785137
1,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.000000,1.200000,0.40000,13.528000,HTC,1.0,0.0,0.0,0.0,2.878561
2,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.000000,1.200000,0.40000,13.528000,HTC,1.0,0.0,0.0,0.0,2.878561
3,cel_18,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,1.345032,1.020847,16.51605,17.562938,cellulose,1.0,0.0,0.0,0.0,2.018886
4,cel_19,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,1.345032,1.020847,16.51605,17.562938,cellulose,1.0,0.0,0.0,0.0,2.324709


In [517]:
fuel_type_counts_after = clean_data['fuel_type'].value_counts()
print(f'Before:{fuel_type_counts}\tAfter: {fuel_type_counts_after}')

Before:fuel_type
sewage           241
digestate        236
wood             236
digestate_car    206
coal             195
digestate_pp     183
digestate_pe     174
ebs2             138
HTC              134
ebs1              62
gummipulver       56
Hemicellulose     44
Lignin            44
cellulose         41
Name: count, dtype: int64	After: fuel_type
sewage           241
wood             236
digestate        233
digestate_car    205
coal             191
digestate_pp     181
digestate_pe     169
ebs2             133
HTC              131
ebs1              60
gummipulver       56
Lignin            44
Hemicellulose     41
cellulose         40
Name: count, dtype: int64


# Train-Test Split


In [518]:
train_data = clean_data[clean_data['fuel_type'] != 'digestate'].drop(columns=['fuel_type']).reset_index(drop=True)
print(clean_data.shape)
train_data.describe()


(1961, 24)


,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,...,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
count,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000,...,1728.000000,1728.000000,1728.000000,1728.000000,1523.000000,1728.000000,1728.000000,1728.000000,1728.000000,1728.000000
mean,797.100694,5.280093,2.420139,920.248843,6.140897,63.647114,14.812557,15.410864,51.176539,6.105416,...,0.651286,0.304686,1.391804,1.704421,21.975900,0.424190,0.110532,0.321181,0.144097,52.568124
std,195.713436,4.402317,3.977279,254.554794,2.490494,13.326686,11.094130,13.065888,13.112625,1.980609,...,0.731605,0.586455,0.362109,5.203061,6.790768,0.494362,0.313643,0.467065,0.351290,23.093297
min,200.000000,0.500000,0.500000,10.000000,2.600000,44.200000,3.064326,0.000000,32.400000,3.265000,...,0.046420,0.000000,0.461416,0.000000,13.528000,0.000000,0.000000,0.000000,0.000000,-2.871983
25%,600.000000,1.000000,1.000000,1000.000000,3.200000,53.700000,6.900000,4.200000,40.305405,4.600000,...,0.146500,0.000000,1.200000,0.200000,17.342580,0.000000,0.000000,0.000000,0.000000,36.228343
50%,800.000000,5.000000,1.000000,1000.000000,5.810000,60.025969,11.550000,11.968500,50.400000,5.700000,...,0.342000,0.000000,1.500000,0.304000,20.130000,0.000000,0.000000,0.000000,0.000000,55.552291
75%,1000.000000,10.000000,1.000000,1000.000000,7.650000,75.400000,16.400000,21.615943,61.534000,7.196250,...,0.800000,0.003570,1.675000,0.400000,30.056920,1.000000,0.000000,1.000000,0.000000,69.460769
max,1200.000000,20.000000,20.000000,1000.000000,10.800000,91.735674,40.900000,37.520000,82.300000,10.132050,...,2.500000,1.585535,1.857000,26.102250,35.800000,1.000000,1.000000,1.000000,1.000000,99.756944


In [519]:
train_data

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,...,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
0,htc_96,200,0.5,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.34200,0.000000,1.200000,0.40000,13.528000,1.0,0.0,0.0,0.0,6.785137
1,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.34200,0.000000,1.200000,0.40000,13.528000,1.0,0.0,0.0,0.0,2.878561
2,htc_152,200,1.0,1.0,1000,7.345,48.2400,6.9000,37.52,33.085000,...,0.34200,0.000000,1.200000,0.40000,13.528000,1.0,0.0,0.0,0.0,2.878561
3,cel_18,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,0.04642,1.345032,1.020847,16.51605,17.562938,1.0,0.0,0.0,0.0,2.018886
4,cel_19,200,1.0,1.0,1000,5.725,87.9005,6.3745,0.00,40.305405,...,0.04642,1.345032,1.020847,16.51605,17.562938,1.0,0.0,0.0,0.0,2.324709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,refuse_derived_101,1170,10.0,1.0,1000,2.700,82.1000,7.0000,8.20,68.700000,...,0.80000,0.000000,1.500000,0.10000,32.177000,0.0,0.0,0.0,1.0,76.923077
1724,refuse_derived_103,1170,10.0,1.0,1000,2.700,82.1000,7.0000,8.20,68.700000,...,0.80000,0.000000,1.500000,0.10000,32.177000,0.0,0.0,0.0,1.0,76.416382
1725,refuse_derived_104,1170,10.0,1.0,1000,2.700,82.1000,7.0000,8.20,68.700000,...,0.80000,0.000000,1.500000,0.10000,32.177000,0.0,0.0,0.0,1.0,73.872119
1726,refuse_derived_114,1180,10.0,1.0,1000,2.700,82.1000,7.0000,8.20,68.700000,...,0.80000,0.000000,1.500000,0.10000,32.177000,0.0,0.0,0.0,1.0,77.795999


In [520]:
test_data = clean_data[
    clean_data['fuel_type'].isin(['digestate'])
].drop(columns=['fuel_type']).reset_index(drop=True)

In [521]:
test_data

,sample,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,...,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic,devol_yield
0,digestate_2,400,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,19.477513
1,digestate_16,400,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,27.310674
2,digestate_300,400,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,19.453481
3,digestate_301,400,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,6.365314
4,digestate_302,400,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,25.649013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,digestate_308,1200,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,59.139403
229,digestate_309,1200,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,60.374640
230,digestate_310,1200,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,54.277600
231,digestate_311,1200,10.0,1.0,1000,10.2,50.7,15.4,23.7,37.5,...,0.2,0.0,1.5,0.4,15.055,0.0,0.0,0.0,1.0,59.020408


In [522]:
X_train = train_data.drop(columns=['sample', 'devol_yield'])
y_train = train_data['devol_yield']

X_test = test_data.drop(columns=['sample', 'devol_yield'])
y_test = test_data['devol_yield']

# kNN Imputation & Standard Scaling

In [523]:
import shap
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

In [524]:
X_train.isnull().sum()

temperature                0
residence_time             0
pressure                   0
heat_rate                  0
wc                         0
vm                         0
fc                         0
ac                         0
c                          0
h                          0
o                          0
n                          0
s                          0
cl                         0
hc                         0
oc                         0
lhv                      205
fuel_category_biomass      0
fuel_category_coal         0
fuel_category_mix          0
fuel_category_plastic      0
dtype: int64

In [525]:
knn_imputer = KNNImputer(n_neighbors=3)
X_train_imputed = knn_imputer.fit_transform(X_train)
X_test_imputed = knn_imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_scaled

,temperature,residence_time,pressure,heat_rate,wc,vm,fc,ac,c,h,...,n,s,cl,hc,oc,lhv,fuel_category_biomass,fuel_category_coal,fuel_category_mix,fuel_category_plastic
0,-3.051776,-1.086127,-0.357166,0.313387,0.483619,-1.156445,-0.713427,1.692616,-1.380103,-1.434528,...,0.270226,-0.422873,-0.519689,-0.529841,-0.250775,-1.336214,1.165090,-0.352517,-0.687856,-0.410313
1,-3.051776,-0.972518,-0.357166,0.313387,0.483619,-1.156445,-0.713427,1.692616,-1.380103,-1.434528,...,0.270226,-0.422873,-0.519689,-0.529841,-0.250775,-1.336214,1.165090,-0.352517,-0.687856,-0.410313
2,-3.051776,-0.972518,-0.357166,0.313387,0.483619,-1.156445,-0.713427,1.692616,-1.380103,-1.434528,...,0.270226,-0.422873,-0.519689,-0.529841,-0.250775,-1.336214,1.165090,-0.352517,-0.687856,-0.410313
3,-3.051776,-0.972518,-0.357166,0.313387,-0.167042,1.820438,-0.760808,-1.179815,-0.829299,-0.801877,...,-1.198866,-0.827006,1.774471,-1.024735,2.847538,-0.706636,1.165090,-0.352517,-0.687856,-0.410313
4,-3.051776,-0.972518,-0.357166,0.313387,-0.167042,1.820438,-0.760808,-1.179815,-0.829299,-0.801877,...,-1.198866,-0.827006,1.774471,-1.024735,2.847538,-0.706636,1.165090,-0.352517,-0.687856,-0.410313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,1.905885,1.072452,-0.357166,0.313387,-1.382012,1.385058,-0.704410,-0.552045,1.336768,1.411380,...,-0.531344,0.203329,-0.519689,0.298880,-0.308450,1.573621,-0.858303,-0.352517,-0.687856,2.437162
1724,1.905885,1.072452,-0.357166,0.313387,-1.382012,1.385058,-0.704410,-0.552045,1.336768,1.411380,...,-0.531344,0.203329,-0.519689,0.298880,-0.308450,1.573621,-0.858303,-0.352517,-0.687856,2.437162
1725,1.905885,1.072452,-0.357166,0.313387,-1.382012,1.385058,-0.704410,-0.552045,1.336768,1.411380,...,-0.531344,0.203329,-0.519689,0.298880,-0.308450,1.573621,-0.858303,-0.352517,-0.687856,2.437162
1726,1.956995,1.072452,-0.357166,0.313387,-1.382012,1.385058,-0.704410,-0.552045,1.336768,1.411380,...,-0.531344,0.203329,-0.519689,0.298880,-0.308450,1.573621,-0.858303,-0.352517,-0.687856,2.437162


# Baseline Models
## Simple Baselines

In [526]:
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor

# Dummy models
mean_model = DummyRegressor(strategy="mean")
median_model = DummyRegressor(strategy="median")

# KNN Regression
knn_model = KNeighborsRegressor(n_neighbors=5)

mean_model.fit(X_train_scaled, y_train)
median_model.fit(X_train_scaled, y_train)
knn_model.fit(X_train_scaled, y_train)
mean_predictions = mean_model.predict(X_test_scaled)
median_predictions = median_model.predict(X_test_scaled)
knn_predictions = knn_model.predict(X_test_scaled)

print('Mean Model Score:', mean_model.score(X_test_scaled, y_test))
print('Median Model Score:', median_model.score(X_test_scaled, y_test))
print('KNN Model Score:', knn_model.score(X_test_scaled, y_test))
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("Mean Model MAE:", mean_absolute_error(y_test, mean_predictions))
print("Median Model MAE:", mean_absolute_error(y_test, median_predictions))
print("KNN Model MAE:", mean_absolute_error(y_test, knn_predictions))

Mean Model Score: -0.0056921677150556516
Median Model Score: -0.11031251579203993
KNN Model Score: -0.9688167342847407
Mean Model MAE: 8.015482527012777
Median Model MAE: 8.543049540137341
KNN Model MAE: 13.720592656021461


## 2. Linear Models
Linear Regression (OLS)

Ridge Regression (L2 regularization)

Lasso Regression (L1 regularization)

Elastic Net (Combination of L1 and L2)

In [527]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

lin_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)
lasso_model = Lasso(alpha=0.1)
elastic_model = ElasticNet(alpha=0.1, l1_ratio=0.5)
lin_model.fit(X_train_scaled, y_train)
ridge_model.fit(X_train_scaled, y_train)
lasso_model.fit(X_train_scaled, y_train)
elastic_model.fit(X_train_scaled, y_train)

print('Linear Model Score:', lin_model.score(X_test_scaled, y_test))
print('Ridge Model Score:', ridge_model.score(X_test_scaled, y_test))
print('Lasso Model Score:', lasso_model.score(X_test_scaled, y_test))
print('Elastic Model Score:', elastic_model.score(X_test_scaled, y_test))

Linear Model Score: -1.666618154355683
Ridge Model Score: -0.2541822627199597
Lasso Model Score: -0.9057315929956664
Elastic Model Score: -0.8099153548945981


## 3. Tree-Based Models
Decision Tree Regression: Simple non-linear model.

Random Forest Regression: Ensemble of decision trees.

Gradient Boosting Regression (GBR)

XGBoost

LightGBM
CatBoost

In [528]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

dt_model = DecisionTreeRegressor(max_depth=5)
rf_model = RandomForestRegressor(n_estimators=100, max_depth=5)
gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)

xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1)
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
dt_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train_scaled, y_train)
gbr_model.fit(X_train_scaled, y_train)
xgb_model.fit(X_train_scaled, y_train)
lgb_model.fit(X_train_scaled, y_train)
dt_predictions = dt_model.predict(X_test_scaled)
gbr_predictions = gbr_model.predict(X_test_scaled)
lgb_predictions = lgb_model.predict(X_test_scaled)
xgb_predictions = xgb_model.predict(X_test_scaled)
rf_predictions = rf_model.predict(X_test_scaled)
print('Decision Tree Regressor Score:', mean_squared_error(y_test, dt_predictions))
print('Gradient Boosting Regressor Score:', mean_squared_error(y_test, gbr_predictions))
print('lightGBM Regressor Score:', mean_squared_error(y_test, lgb_predictions))
print('xgboost Regressor Score:', mean_squared_error(y_test, xgb_predictions))
print('RF Regressor Score:', mean_squared_error(y_test, rf_predictions))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1466
[LightGBM] [Info] Number of data points in the train set: 1728, number of used features: 21
[LightGBM] [Info] Start training from score 52.568124
Decision Tree Regressor Score: 136.90103075126387
Gradient Boosting Regressor Score: 114.54514221855264
lightGBM Regressor Score: 157.58203750185197
xgboost Regressor Score: 138.64143222898116
RF Regressor Score: 108.23466612892392


In [529]:
print('Decision Tree Regressor Score:', dt_model.score(X_test_scaled, y_test))
print('Gradient Boosting Regressor Score:', gbr_model.score(X_test_scaled, y_test))
print('lightGBM Regressor Score:', lgb_model.score(X_test_scaled, y_test))
print('XGBoost Regressor Score:', xgb_model.score(X_test_scaled, y_test))
print('RF Regressor Score:', rf_model.score(X_test_scaled, y_test))

Decision Tree Regressor Score: -0.012902293006961107
Gradient Boosting Regressor Score: 0.15250428306282648
lightGBM Regressor Score: -0.16591676663370336
XGBoost Regressor Score: -0.02577916206965969
RF Regressor Score: 0.19919418500201636


## 4. Nonlinear & Probabilistic Models
Gaussian Process Regression (GPR)

Support Vector Regression (SVR)

Neural Networks (MLP)

In [530]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

gpr_model = GaussianProcessRegressor()
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', max_iter=2000)
gpr_model.fit(X_train_scaled, y_train)
svr_model.fit(X_train_scaled, y_train)
mlp_model.fit(X_train_scaled, y_train)
print('Gaussian Process Regressor Score:', gpr_model.score(X_test_scaled, y_test))
print('SVR Score:', svr_model.score(X_test_scaled, y_test))
print('MLP Score:', mlp_model.score(X_test_scaled, y_test))

Gaussian Process Regressor Score: -35.99637734192186
SVR Score: -0.032725647774969335
MLP Score: 0.14321418380741158
